In [172]:
'''
!pip install ipycanvas
!pip install pandas
'''

In [12]:
import numpy as np
import pandas as pd
from ipycanvas import MultiCanvas, Canvas, hold_canvas

In [13]:
#Write program manualy
#                1  0  b  R  A  B  C
program_memory= [[0, 0, 1, 0, 0, 1, 0], #   1
                 [1, 0, 0, 0, 1, 0, 0], # A 0
                 [0, 0, 1, 0, 0, 0, 1], #   b
                 
                 [1, 0, 0, 0, 0, 1, 0], #   1
                 [0, 1, 0, 1, 1, 0, 0], # B 0
                 [0, 0, 0, 0, 0, 0, 0], #   b
                 
                 [0, 1, 0, 1, 0, 0, 1], #   1
                 [0, 0, 1, 0, 1, 0, 0], # C 0
                 [1, 0, 0, 1, 0, 0, 1]] #   b

In [14]:
#Import program

In [15]:
#Write data tape manualy
input_tape='001b001b001b00010b110b0b110b'
data_tape=list(input_tape)

In [16]:
'''
#Import data tape
data_tape=list(open('Input Tape.txt').read())
print(''.join(input_tape))
'''

"\n#Import data tape\ndata_tape=list(open('Input Tape.txt').read())\nprint(''.join(input_tape))\n"

In [17]:
#Initial setup
initial_state='A'
initial_tape_position=15
iterations=93

In [18]:
#Create output images
block_size=20
multi_canvas = MultiCanvas(2, width=block_size*(len(data_tape)+7), height=block_size*(iterations+2))
multi_canvas[0].fill_style = 'black'
multi_canvas[0].fill_rect(0, 0, block_size*(len(data_tape)+7), height=block_size*(iterations+2))

#Setup font
multi_canvas[1].font = '18px Lucida Console '
multi_canvas[1].fill_style = '#00ff00'
text_offset_x=4
text_offset_y=15

#create blocks to fill the canvas
canvas_1=Canvas(height=block_size, width=block_size)
canvas_1.fill_style='#00cc00'
canvas_1.fill_rect(1, 1, width=block_size-2, height=block_size-2)
canvas_0=Canvas(height=block_size, width=block_size)
canvas_0.fill_style='#3366ff'
canvas_0.fill_rect(1, 1, width=block_size-2, height=block_size-2)
canvas_b=Canvas(height=block_size, width=block_size)
canvas_b.fill_style='yellow'
canvas_b.fill_rect(1, 1, width=block_size-2, height=block_size-2)
canvas_head=Canvas(height=block_size,width=block_size)
head_size=8
canvas_head.fill_style='black'
canvas_head.fill_rect((block_size-head_size)/2, (block_size-head_size)/2, width=head_size, height=head_size)

#multi_canvas[1].draw_image(canvas_1, 0, 0)
#multi_canvas[1].draw_image(canvas_head, 0, 0)
#multi_canvas[1].fill_text('A', 2, 15)
multi_canvas

MultiCanvas(height=1900)

In [19]:
#Create variables to save reader head position and instrction row during iterations
if initial_state=='A':
    current_instruction=0
elif initial_state=='B':
    current_instruction=3
elif initial_state=='C':
    current_instruction=6
current_tape_position=initial_tape_position

last_write='X'

#Output history table
output_history=pd.DataFrame([['0','0',0,'0']],columns=['State','Data Tape','Reader head position','Symbol read'])

In [20]:
#Controler code
for i in range(0,iterations):   
    #save result to dataframe (Output: Iteration | State | Data Tape | Reader head position | Symbol)
    if 0<=current_instruction<=2:
        output_history.at[i,'State']= 'A'    
    if 3<=current_instruction<=5:
        output_history.at[i,'State']= 'B'  
    if 6<=current_instruction<=8:
        output_history.at[i,'State']= 'C' 
    output_history.at[i,'Data Tape']= ''.join(data_tape)
    output_history.at[i,'Reader head position']=current_tape_position
    output_history.at[i,'Symbol read']=input_tape[current_tape_position]
    
    #Draw to output image
    with hold_canvas(multi_canvas[1]):
        if 0<=current_instruction<=2:
            multi_canvas[1].fill_text('A', text_offset_x+block_size*3, text_offset_y+(i+1)*block_size)
        if 3<=current_instruction<=5:
            multi_canvas[1].fill_text('B', text_offset_x+block_size*3, text_offset_y+(i+1)*block_size)
        if 6<=current_instruction<=8:
            multi_canvas[1].fill_text('C', text_offset_x+block_size*3, text_offset_y+(i+1)*block_size)
        for j in range(0,len(data_tape)):
            if data_tape[j]=='1':
                multi_canvas[1].draw_image(canvas_1, block_size*(4+j), block_size*(i+1))
            if data_tape[j]=='0':
                multi_canvas[1].draw_image(canvas_0, block_size*(4+j), block_size*(i+1))
            if data_tape[j]=='b':
                multi_canvas[1].draw_image(canvas_b, block_size*(4+j), block_size*(i+1))
            if j==current_tape_position:
                multi_canvas[1].draw_image(canvas_head, block_size*(4+j), block_size*(i+1))     
        multi_canvas[1].fill_text(i, text_offset_x+block_size*1, text_offset_y+(i+1)*block_size)
        multi_canvas[1].fill_text('✎'+last_write, text_offset_x+block_size*(len(data_tape)+4), text_offset_y+(i+1)*block_size)
    
    #Read and find current instruction
    if data_tape[current_tape_position] == '1':
        current_instruction=current_instruction
    elif data_tape[current_tape_position] == '0':
        current_instruction=current_instruction+1
    elif data_tape[current_tape_position] == 'b':
        current_instruction=current_instruction+2
        
    #write to tape
    if program_memory[current_instruction][0]:
        data_tape[current_tape_position]='1'
    if program_memory[current_instruction][1]:
        data_tape[current_tape_position]='0'
    if program_memory[current_instruction][2]:
        data_tape[current_tape_position]='b'
    last_write=data_tape[current_tape_position] #saved for image output next iteration
    
    #move tape
    if program_memory[current_instruction][3]:
        current_tape_position+= 1
    else:
        current_tape_position-= 1

    #change state
    if program_memory[current_instruction][4]:
        current_instruction=0
    elif program_memory[current_instruction][5]:
        current_instruction=3
    elif program_memory[current_instruction][6]:
        current_instruction=6
    else:
        current_instruction=0